In [1]:
import pandas as pd
import pandasql as ps
from pandasql import *
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import glob
import warnings
from datetime import datetime
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from functools import reduce
from scipy.stats.mstats import gmean

warnings.simplefilter("ignore")

# For the widget

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

date = 'Apr 28'
vol_date = '2023-04-28'

valid_expiry = [
    "2010-11-25", "2010-12-30", "2011-01-27", "2011-02-24", "2011-03-31", "2011-04-28", 
    "2011-05-26", "2011-06-30", "2011-07-28", "2011-08-25", "2011-09-29", "2011-10-25", 
    "2011-11-24", "2011-12-29", "2012-01-25", "2012-02-23", "2012-03-29", "2012-04-26", 
    "2012-05-31", "2012-06-28", "2012-07-26", "2012-08-30", "2012-09-27", "2012-10-25", 
    "2012-11-29", "2012-12-27", "2013-01-31", "2013-02-28", "2013-03-28", "2013-04-25", 
    "2013-05-30", "2013-06-27", "2013-07-25", "2013-08-29", "2013-09-26", "2013-10-31", 
    "2013-11-28", "2013-12-26", "2014-01-30", "2014-02-26", "2014-02-27", "2014-03-27", 
    "2014-04-24", "2014-05-29", "2014-06-26", "2014-07-31", "2014-08-28", "2014-09-25", 
    "2014-10-30", "2014-11-27", "2014-12-24", "2015-01-29", "2015-02-26", "2015-03-26", 
    "2015-04-30", "2015-05-28", "2015-06-25", "2015-07-30", "2015-08-27", "2015-09-24", 
    "2015-10-29", "2015-11-26", "2015-12-31", "2016-01-28", "2016-02-25", "2016-03-31", 
    "2016-04-28", "2016-05-26", "2016-06-30", "2016-07-28", "2016-08-25", "2016-09-29", 
    "2016-10-27", "2016-11-24", "2016-12-29", "2017-01-25", "2017-02-23", "2017-03-30", 
    "2017-04-27", "2017-05-25", "2017-06-29", "2017-07-27", "2017-08-31", "2017-09-28", 
    "2017-10-26", "2017-11-30", "2017-12-28", "2018-01-25", "2018-02-22", "2018-03-28", 
    "2018-03-29", "2018-04-26", "2018-05-31", "2018-06-28", "2018-07-26", "2018-08-30", 
    "2018-09-27", "2018-10-25", "2018-11-29", "2018-12-27", "2019-01-31", "2019-02-28", 
    "2019-03-28", "2019-04-25", "2019-05-30", "2019-06-27", "2019-07-25", "2019-08-29", 
    "2019-09-26", "2019-10-31", "2019-11-28", "2019-12-26", "2020-01-30", "2020-02-27", 
    "2020-03-26", "2020-04-30", "2020-05-28", "2020-06-25", "2020-07-30", "2020-08-27", 
    "2020-09-24", "2020-10-29", "2020-11-26", "2020-12-31", "2021-01-28", "2021-02-25", 
    "2021-03-25", "2021-04-29", "2021-05-27", "2021-06-24", "2021-07-29", "2021-08-26", 
    "2021-09-30", "2021-10-28", "2021-11-25", "2021-12-30", "2022-01-27", "2022-02-24", 
    "2022-03-31", "2022-04-28", "2022-05-26", "2022-06-30", "2022-07-28", "2022-08-25",
    "2022-09-29", "2022-10-27", "2022-11-24", "2022-12-29", "2023-01-25", "2023-02-23", 
    "2023-03-29", "2023-04-27", "2023-05-25", "2023-06-29"
]

In [2]:
# Get NIFTY options data

path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/bhav/options_02_split/NIFTY/Apr 28/Merged/iv_greeks.csv'
df = pd.read_csv(path)

In [3]:
# ATM Option prices data for 'days to expiry' that are between 14 and 42
dte_14_42_df = df[
    (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
#     & ((df['strike'] % 100) == 0)
].reset_index().drop('index', axis=1)
dte_14_42_df['date_expiry'] = dte_14_42_df['date'] + '_' + dte_14_42_df['expiry']
dte_14_42_df['moneyness_abs'] = dte_14_42_df['moneyness_close'].abs()
temp_grouped = dte_14_42_df.groupby('date_expiry')['moneyness_abs'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
dte_14_42_df = dte_14_42_df[[
    'date_expiry', 'date', 'expiry', 'days_to_expiry', 'moneyness_abs', 'strike', 'implied_volatility', 
    'put_close', 'call_close', 'forward_close', 'close_ul', 'delta', 'gamma', 'theta', 'vega'
]]
dte_14_42_atm_df = temp_grouped_df.merge(dte_14_42_df, on=['date_expiry', 'moneyness_abs'], how='inner')
dte_14_42_atm_df
temp_grouped = dte_14_42_atm_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
dte_14_42_atm_df = temp_grouped_df.merge(dte_14_42_atm_df, on=['date', 'days_to_expiry'], how='inner')

In [4]:
# Get volatility data
path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/vol_estimation_new/NIFTY/2023-04-28.csv'
rvol_df = pd.read_csv(path)
rvol_reduced_df = rvol_df[[
    'date', '20d_c2c_vol'
]]

In [5]:
# Merge ATM option data with the volatility data
atm_rvol_df = dte_14_42_atm_df.merge(rvol_reduced_df, on=['date'], how='inner')

In [6]:
# Drop NA from the atm_rvol_df DF
atm_rvol_improved_df = atm_rvol_df.dropna().reset_index().drop('index', axis=1)

In [7]:
# ATM iv_rv_ratio and Volatility Risk Premium (vrp) for atm_rvol_improved_df
atm_rvol_improved_df['iv_rv_ratio'] = atm_rvol_improved_df['implied_volatility'] / atm_rvol_improved_df['20d_c2c_vol']
atm_rvol_improved_df['vrp'] = atm_rvol_improved_df['iv_rv_ratio'] - 1

In [8]:
# Separate out the nearest expiry
temp_df = atm_rvol_improved_df.set_index(['date', 'expiry'])
vrp_month_0_df = temp_df.loc[
    (slice(None) , atm_rvol_improved_df['expiry']), 
    ['days_to_expiry', 'vrp']
].reset_index()
vrp_month_0_df = vrp_month_0_df[
    vrp_month_0_df['date'] >= '2016-01-01'
].reset_index().drop('index', axis=1)

In [9]:
# 75th percentile Volatility Risk Premium (VRP) value
def iv_rv_ratio_0_mny(ul, dte, rv_type, pc_date):
    path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/IVol PC Values/Date Wise IVol Surface/0/'+ul+'/Concatenated/iv_concat.csv'
    df = pd.read_csv(path)
    df = df.sort_values(by=['date'], ascending=[True])
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    df = df.reset_index().drop('index', axis=1)
    iv_df = df[df['date'] <= pc_date][['date', str(dte)]]
    path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/vol_estimation_new/'+ul+'/'+vol_date+'.csv'
    df = pd.read_csv(path)
    rv_df = df[[
        'date', 'close_ul', 'log_returns', '20d_c2c_vol'
    ]]
    iv_rv_df = rv_df.merge(iv_df, on=['date'], how='inner')
    iv_rv_df['vrp_20d_c2c'] = iv_rv_df[str(dte)] / iv_rv_df['20d_c2c_vol'] - 1
    vrp_pc.append(iv_rv_df['vrp_20d_c2c'].quantile(0.75, interpolation='nearest'))
    
dates = [
    '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', 
    '2021-12-31', '2022-12-31'
]
dtes = list(range(14, 43))
ul = 'NIFTY'
rv_type = '20d_c2c'
vrp_pc_df = pd.DataFrame(columns=['date'])
for dte in dtes:
    vrp_pc_df[dte] = 0
vrp_pc_df['date'] = dates
vrp_pc_df = vrp_pc_df.set_index('date')
for pc_date in dates:
    vrp_pc = []
    for dte in dtes:
        iv_rv_ratio_0_mny(ul, dte, rv_type, pc_date)
    vrp_pc_df.loc[pc_date] = vrp_pc

In [10]:
# Set up vrp_month_0_df so that the 75th percentile vrp value can be used to determine the 
# required action.
dtes = vrp_month_0_df['days_to_expiry'].to_list()
c1 = (vrp_month_0_df['date']) <= '2016-12-31'
c2 = (vrp_month_0_df['date']) <= '2017-12-31'
c3 = (vrp_month_0_df['date']) <= '2018-12-31'
c4 = (vrp_month_0_df['date']) <= '2019-12-31'
c5 = (vrp_month_0_df['date']) <= '2020-12-31'
c6 = (vrp_month_0_df['date']) <= '2021-12-31'
c7 = (vrp_month_0_df['date']) <= '2022-12-31'
c8 = (vrp_month_0_df['date']) <= '2023-12-31'
pc_date = [
    '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', 
    '2021-12-31', '2022-12-31'
]
default = 0
vrp_month_0_df['pc_date'] = np.select([c1, c2, c3, c4, c5, c6, c7, c8], pc_date, default=default)
vrp_month_0_df['vrp_50pc_value'] = ''
for dte, i in zip(dtes, range(0, len(vrp_month_0_df))):
    vrp_month_0_df['vrp_50pc_value'][i] = vrp_pc_df.loc[vrp_month_0_df['pc_date'][i]][dte]
vrp_month_0_df = vrp_month_0_df.drop_duplicates(subset=['date'], keep='last')

In [11]:
# Mark whether a short volatility trade will taken or not, by comparing 'vrp' column
# with 'vrp_50pc_value' column
vrp_month_0_df['trade_decision'] = np.where(vrp_month_0_df['vrp'] >= vrp_month_0_df['vrp_50pc_value'], 1, 0)

In [12]:
# Merge vrp_month_0_df DF and atm_rvol_improved_df
atm_df = vrp_month_0_df.merge(atm_rvol_improved_df, on=['date', 'expiry', 'days_to_expiry', 'vrp'], how='inner')

In [13]:
# Multiple operations
# Straddle price, quantities to trade (which is 1 Crore / NIFTY spot), etc.
atm_df['straddle_price'] = atm_df['put_close'] + atm_df['call_close']
atm_df['close_ul'] = pd.to_numeric(atm_df['close_ul'], errors='coerce')
atm_df['trade_quantity'] = round(np.floor(10000000.00 / atm_df['close_ul'])/100)*100
atm_df = atm_df.reset_index().drop('index', axis=1)

In [14]:
# Call delta and put delta
atm_df['call_delta'] = np.where(atm_df['delta'] > 0, atm_df['delta'], 1 + atm_df['delta'])
atm_df['put_delta'] = np.where(atm_df['delta'] < 0, atm_df['delta'], atm_df['delta'] - 1)
atm_df['position_delta'] = -1 * (atm_df['call_delta'] + atm_df['put_delta']) * atm_df['trade_quantity']
atm_df['position_gamma'] = -1 * (atm_df['gamma'] * 2) * atm_df['trade_quantity']
atm_df['position_theta'] = -1 * (atm_df['theta'] * 2) * atm_df['trade_quantity']
atm_df['position_vega'] = -1 * (atm_df['vega'] * 2) * atm_df['trade_quantity']

In [15]:
# Rename columns in atm_df to make it easier while merging with otm_df
atm_df = atm_df.rename(columns={
    'strike': 'atm_strike',
    'implied_volatility': 'atm_iv', 
    'put_close': 'atm_put_price',
    'call_close': 'atm_call_price',
    'delta': 'atm_delta',
    'gamma': 'atm_gamma',
    'theta': 'atm_theta',
    'vega': 'atm_vega',
    'straddle_price': 'atm_straddle_price',
    'trade_quantity': 'atm_qty',
    'call_delta': 'atm_call_delta',
    'put_delta': 'atm_put_delta',
    'position_delta': 'atm_position_delta',
    'position_gamma': 'atm_position_gamma',
    'position_theta': 'atm_position_theta',
    'position_vega': 'atm_position_vega'
})

In [16]:
# Separate out individual trades
# Definition of a trade:
# Consecutive rows with trade_decisions column being equal to 1 is a trade.
trade_group_df = pd.DataFrame(columns=['date', 'expiry', 'trade_decision', 'trade_group'])
trade_group_count = 1
expiries = atm_df['expiry'].unique()
for expiry in expiries:
    temp_expiry_df = atm_df[atm_df['expiry'] == expiry][[
        'date', 'expiry', 'trade_decision'
    ]].reset_index().drop('index', axis=1)
    temp_expiry_df['trade_group'] = ''
    for i in range(0, len(temp_expiry_df)):
        if i == 0:
            if (temp_expiry_df['trade_decision'][i] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
            else:
                temp_expiry_df['trade_group'][i] = 0
        elif (i > 0) & (i < len(temp_expiry_df)-1):
            if (temp_expiry_df['trade_decision'][i-1] == 1) & (temp_expiry_df['trade_decision'][i] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
            elif (temp_expiry_df['trade_decision'][i-1] == 1) & (temp_expiry_df['trade_decision'][i] == 0):
                temp_expiry_df['trade_group'][i] = trade_group_count
                trade_group_count = trade_group_count + 1
            elif (temp_expiry_df['trade_decision'][i-1] == 0) & (temp_expiry_df['trade_decision'][i] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
            else:
                temp_expiry_df['trade_group'][i] = 0
        else:
            if (temp_expiry_df['trade_decision'][i-1] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
                trade_group_count = trade_group_count + 1
            else:
                temp_expiry_df['trade_group'][i] = 0
    trade_group_count = trade_group_count + 1
    trade_group_df = pd.concat([trade_group_df, temp_expiry_df], axis=0)
    
trade_group_df = trade_group_df.drop('trade_decision', axis=1)
atm_df = atm_df.merge(trade_group_df, on=['date', 'expiry'], how='inner')

In [17]:
# Define the hedges for the short volatility trade - put leg
put_otm_df = df[
    (df['moneyness_close'] <= 0.11)
    & (df['moneyness_close'] >= 0.05)
    & (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
    & (df['date'] >= '2016-01-01')
    & ((df['strike'] % 100) == 0)
][[
    'date', 'expiry', 'strike', 'days_to_expiry', 'otm_option_type', 'moneyness_close', 'close_option', 
    'implied_volatility', 'delta', 'gamma', 'theta', 'vega'
]]
put_otm_df['moneyness_abs'] = put_otm_df['moneyness_close'].abs()
temp = put_otm_df.groupby(['date', 'expiry'])['moneyness_abs'].max()
temp_df = pd.DataFrame(temp)
put_otm_lowest_mny_df = put_otm_df.merge(temp_df, on=['date', 'expiry', 'moneyness_abs'], how='inner')
put_otm_lowest_mny_df = put_otm_lowest_mny_df.rename(columns = {
    'strike': 'otm_put_strike',
    'otm_option_type': 'otm_put_option_type',
    'moneyness_close': 'otm_put_moneyness',
    'close_option': 'otm_put_price',
    'implied_volatility': 'otm_put_iv',
    'delta': 'otm_put_delta',
    'gamma': 'otm_put_gamma',
    'theta': 'otm_put_theta',
    'vega' : 'otm_put_vega'
})
put_otm_lowest_mny_df = put_otm_lowest_mny_df[[
    'date', 'expiry', 'days_to_expiry', 'otm_put_strike', 'otm_put_option_type', 'otm_put_moneyness', 
    'otm_put_price', 'otm_put_iv', 'otm_put_delta', 'otm_put_gamma', 'otm_put_theta',
    'otm_put_vega'
]]
temp_grouped = put_otm_lowest_mny_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
put_otm_lowest_mny_df = temp_grouped_df.merge(put_otm_lowest_mny_df, on=['date', 'days_to_expiry'], how='inner')
put_otm_lowest_mny_df = put_otm_lowest_mny_df.drop_duplicates(
    subset=['date'], keep='last', ignore_index=True
)

In [18]:
# Define the hedges for the short volatility trade - call leg
call_otm_df = df[
    (df['moneyness_close'] <= -0.01)
    & (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
    & (df['date'] >= '2016-01-01')
    & ((df['strike'] % 100) == 0)
][[
    'date', 'expiry', 'strike', 'days_to_expiry', 'otm_option_type', 'moneyness_close', 'close_option', 
    'implied_volatility', 'delta', 'gamma', 'theta', 'vega'
]]
call_otm_df['moneyness_abs'] = call_otm_df['moneyness_close'].abs()
call_otm_df = call_otm_df.merge(put_otm_lowest_mny_df, on=['date', 'expiry', 'days_to_expiry'], how='left')
call_otm_df = call_otm_df[
    (call_otm_df['vega'] <= call_otm_df['otm_put_vega'])
]
temp = call_otm_df.groupby(['date', 'expiry'])['vega'].max()
temp_df = pd.DataFrame(temp)
call_otm_lowest_mny_df  = call_otm_df.merge(temp_df, on=['date', 'expiry', 'vega'], how='inner')
call_otm_lowest_mny_df = call_otm_lowest_mny_df.rename(columns = {
    'strike': 'otm_call_strike',
    'otm_option_type': 'otm_call_option_type',
    'moneyness_close': 'otm_call_moneyness',
    'close_option': 'otm_call_price',
    'implied_volatility': 'otm_call_iv',
    'delta': 'otm_call_delta',
    'gamma': 'otm_call_gamma',
    'theta': 'otm_call_theta',
    'vega' : 'otm_call_vega'
})
call_otm_lowest_mny_df = call_otm_lowest_mny_df[[
    'date', 'expiry', 'days_to_expiry', 'otm_call_strike', 'otm_call_option_type', 'otm_call_moneyness', 
    'otm_call_price', 'otm_call_iv', 'otm_call_delta', 'otm_call_gamma', 'otm_call_theta',
    'otm_call_vega'
]]
temp_grouped = call_otm_lowest_mny_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
call_otm_lowest_mny_df = temp_grouped_df.merge(call_otm_lowest_mny_df, on=['date', 'days_to_expiry'], how='inner')
call_otm_lowest_mny_df = call_otm_lowest_mny_df.drop_duplicates(
    subset=['date'], keep='last', ignore_index=True
)

In [19]:
# Merge put_otm_df and call_otm_df
otm_df = put_otm_lowest_mny_df.merge(call_otm_lowest_mny_df, on=['date', 'expiry', 'days_to_expiry'], how='inner')

In [20]:
# Merge atm_df with otm_df
atm_otm_df = atm_df.merge(otm_df, on=['date', 'expiry', 'days_to_expiry'], how='inner')

In [21]:
# Filter out wrong values of iv, both atm and otm
atm_otm_df = atm_otm_df[
    (atm_otm_df['atm_iv'].between(0.05, 5))
    & (atm_otm_df['otm_put_iv'].between(0.05, 5))
    & (atm_otm_df['otm_call_iv'].between(0.05, 5))
].reset_index().drop('index', axis=1)

In [22]:
# Calculate the quantities of otm puts and calls
atm_otm_df['otm_put_qty'] = 1
atm_otm_df['otm_call_qty'] = 1
atm_otm_df['otm_put_position_qty_temp'] = np.ceil(atm_otm_df['atm_qty'] * atm_otm_df['otm_put_qty']/100)*100
atm_otm_df['otm_call_position_qty_temp'] = np.ceil(atm_otm_df['atm_qty'] * atm_otm_df['otm_call_qty']/100)*100
atm_otm_df['otm_put_position_qty'] = atm_otm_df[['atm_qty', 'otm_put_position_qty_temp']].max(axis=1)
atm_otm_df['otm_call_position_qty'] = atm_otm_df[['atm_qty', 'otm_call_position_qty_temp']].max(axis=1)

In [23]:
# Calculate over greeks for the position
# Remove unnecessary columns
atm_otm_df['position_delta'] = -1 * (atm_otm_df['atm_call_delta'] + atm_otm_df['atm_put_delta']) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_delta'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_delta'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_gamma'] = -1 * (atm_otm_df['atm_gamma'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_gamma'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_gamma'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_theta'] = -1 * (atm_otm_df['atm_theta'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_theta'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_theta'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_vega'] = -1 * (atm_otm_df['atm_vega'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_vega'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_vega'] * atm_otm_df['otm_put_position_qty'] / (1)

# Adjusting for "residual" delta when entering into the trade
atm_otm_df['additional_otm_put_delta'] = ''
atm_otm_df['additional_otm_call_delta'] = ''
c1 = (atm_otm_df['position_delta']) < 0
c2 = (atm_otm_df['position_delta']) > 0
c3 = (atm_otm_df['position_delta']) == 0
additional_otm_put_delta = [0, atm_otm_df['position_delta'], 0]
additional_otm_call_delta = [- atm_otm_df['position_delta'], 0, 0]
default = 0
atm_otm_df['additional_otm_put_delta'] = np.select([c1, c2, c3], additional_otm_put_delta, default=default)
atm_otm_df['additional_otm_call_delta'] = np.select([c1, c2, c3], additional_otm_call_delta, default=default)

# atm_otm_df['otm_put_position_qty'] = - atm_otm_df['additional_otm_put_delta'] / atm_otm_df['otm_put_delta'] + atm_otm_df['otm_put_position_qty']
# atm_otm_df['otm_call_position_qty'] = atm_otm_df['additional_otm_call_delta'] / atm_otm_df['otm_call_delta'] + atm_otm_df['otm_call_position_qty']

atm_otm_df['otm_put_position_qty'] = atm_otm_df['atm_qty']
atm_otm_df['otm_call_position_qty'] = atm_otm_df['atm_qty']

atm_otm_df['position_delta'] = -1 * (atm_otm_df['atm_call_delta'] + atm_otm_df['atm_put_delta']) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_delta'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_delta'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_gamma'] = -1 * (atm_otm_df['atm_gamma'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_gamma'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_gamma'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_theta'] = -1 * (atm_otm_df['atm_theta'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_theta'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_theta'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_vega'] = -1 * (atm_otm_df['atm_vega'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_vega'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_vega'] * atm_otm_df['otm_put_position_qty'] / (1)

atm_otm_improved_df = atm_otm_df[[
    'date', 'expiry', 'days_to_expiry', 'trade_decision', 'trade_group', 'forward_close', 
    'atm_strike',  'atm_put_price', 'atm_call_price', 'close_ul', 'atm_iv', '20d_c2c_vol', 'vrp',
    'atm_delta', 'atm_gamma', 'atm_theta', 'atm_vega', 
    'atm_straddle_price', 'atm_qty', 'atm_call_delta', 'atm_put_delta', 
    'atm_position_delta', 'otm_put_strike', 'otm_put_price', 'otm_put_iv', 'otm_put_delta', 
    'otm_put_gamma', 'otm_put_theta', 'otm_put_vega', 'otm_call_strike', 'otm_call_price',
    'otm_call_iv', 'otm_call_delta', 'otm_call_gamma', 'otm_call_theta', 'otm_call_vega', 
    'otm_put_qty', 'otm_call_qty', 'otm_put_position_qty', 'otm_call_position_qty',
    'position_delta', 'position_gamma', 'position_theta', 'position_vega'
]].rename(columns={
    '20d_c2c_vol': 'realized_vol',
    'vrp': 'vol_risk_premium'
})

In [24]:
# Standard deviation of 1-day logarithmic changes in 1-month implied volatility 
# "vol_of_vol" or "vol_of_iv"
def vol_of_iv(ul, dte):
    path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/IVol PC Values/Date Wise IVol Surface/0/'+ul+'/Concatenated/iv_concat.csv'
    df = pd.read_csv(path)
    iv_df = df[['date', str(dte)]].sort_values(by=['date'], ascending=True).reset_index().drop('index', axis=1)
    iv_df = iv_df.dropna()
    iv_df[str(dte)] = iv_df[str(dte)].fillna(0)
    iv_df = iv_df[iv_df[str(dte)] != 0].reset_index().drop('index', axis=1)
    iv_df = iv_df.rename(columns={
        str(dte): '30_day_iv'
    })
    iv_df['log_iv_returns'] = np.log(iv_df['30_day_iv'] / iv_df['30_day_iv'].shift(1))
    iv_df['vol_of_iv_returns'] = iv_df['log_iv_returns'].rolling(20).std()
    iv_df['mean_of_iv_returns'] = iv_df['log_iv_returns'].mean()
    vol_of_iv_returns = iv_df['vol_of_iv_returns'].iloc[len(iv_df) - 1]
    mean_of_iv_returns = iv_df['mean_of_iv_returns'].iloc[len(iv_df) - 1]
    return mean_of_iv_returns, vol_of_iv_returns

ul = ['NIFTY']
dte = 30

for ul in ul:
    mean_of_iv_returns, vol_of_iv_returns = vol_of_iv(ul, dte)

In [25]:
# Calculate position value
atm_otm_improved_df['position_value'] = (- atm_otm_improved_df['atm_straddle_price'] * atm_otm_improved_df['atm_qty']) + (atm_otm_improved_df['otm_put_price'] * atm_otm_improved_df['otm_put_position_qty']) + (atm_otm_improved_df['otm_call_price'] * atm_otm_improved_df['otm_call_position_qty'])

In [26]:
# Gamma risk and Vega risk
atm_otm_improved_df['gamma_risk'] = 0.5 * atm_otm_improved_df['position_gamma'] * (atm_otm_improved_df['realized_vol'] / math.sqrt(252) * atm_otm_improved_df['forward_close']) ** 2
atm_otm_improved_df['vega_risk'] = 100*atm_otm_improved_df['atm_iv'] * (math.e**(vol_of_iv_returns)-1) * atm_otm_improved_df['position_vega']

In [27]:
# PnL calculation for the short vol trades
# Delta adjustments
trade_groups = list(set(atm_otm_improved_df['trade_group'].unique()) - set([0]))
# trade_groups = [2, 3, 9]
short_pnl_df = pd.DataFrame(columns=[
    'date', 'options_pnl', 'futures_pnl', 'atm_pnl'
])
daily_greek_exposure_df = pd.DataFrame(columns=[
    'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close', 'atm_iv', 
    'atm_delta', 'atm_qty', 'atm_call_delta', 'atm_put_delta', 'atm_gamma', 'atm_theta', 
    'atm_vega', 'otm_put_strike', 'otm_put_position_qty', 'otm_put_price', 
    'otm_put_iv', 'otm_put_delta', 'otm_put_gamma', 'otm_put_theta', 'otm_put_vega',
    'otm_call_strike', 'otm_call_position_qty', 'otm_call_price', 
    'otm_call_iv', 'otm_call_delta', 'otm_call_gamma', 'otm_call_theta', 'otm_call_vega',
    'position_delta', 'net_delta', 'futures_qty_to_buy_or_sell', 
    'net_futures_qty_post_hedging', 'net_delta_post_hedging', 'futures_times_qty'
    'position_gamma', 'position_theta', 'position_vega'
])
delta_adjustments_df = pd.DataFrame(columns=[
    'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close', 'atm_delta', 
    'atm_qty', 'atm_call_delta', 'atm_put_delta', 'otm_put_strike', 'otm_put_price', 
    'otm_put_position_qty', 'otm_put_delta', 'otm_call_strike', 'otm_call_price', 
    'otm_call_position_qty', 'otm_call_delta', 
    'position_delta', 'net_delta', 'futures_qty_to_buy_or_sell', 
    'net_futures_qty_post_hedging', 'net_delta_post_hedging', 'futures_times_qty'
])
date_list = []
options_pnl_list = []
futures_pnl_list = []
net_pnl_list = []
for tg in trade_groups:
    trade_df = pd.DataFrame(columns=[
        'date', 'expiry', 'atm_strike', 'forward_close', 'atm_iv', 'atm_delta', 'atm_gamma', 
        'atm_theta', 'atm_vega',
        'atm_qty', 'otm_put_price', 'otm_put_position_qty', 'otm_put_iv', 'otm_put_delta',
        'otm_put_gamma', 'otm_put_theta', 'otm_put_vega',
        'otm_call_price', 'otm_call_position_qty', 'otm_call_iv', 'otm_call_delta', 
        'otm_call_gamma', 'otm_call_theta', 'otm_call_vega'
    ])
    temp_df = atm_otm_improved_df[atm_otm_improved_df['trade_group'] == tg][[
        'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close', 'atm_iv',
        'atm_delta', 'atm_gamma', 'atm_theta', 'atm_vega', 'atm_qty', 
        'otm_put_strike', 'otm_put_price', 'otm_put_position_qty', 'otm_put_iv',
        'otm_put_delta', 'otm_put_gamma', 'otm_put_theta', 'otm_put_vega',
        'otm_call_strike', 'otm_call_price', 'otm_call_position_qty', 'otm_call_iv',
        'otm_call_delta', 'otm_call_gamma', 'otm_call_theta', 'otm_call_vega'
    ]].reset_index().drop('index', axis=1)
    atm_strike = temp_df['atm_strike'].iloc[0]
    otm_put_strike = temp_df['otm_put_strike'].iloc[0]
    otm_call_strike = temp_df['otm_call_strike'].iloc[0]
    expiry = temp_df['expiry'].iloc[0]
    atm_qty = temp_df['atm_qty'].iloc[0]
    otm_put_position_qty = temp_df['otm_put_position_qty'].iloc[0]
    otm_call_position_qty = temp_df['otm_call_position_qty'].iloc[0]
    trade_df_date = []
    trade_df_expiry = []
    trade_df_atm_strike = []
    trade_df_atm_straddle_price = []
    trade_df_forward_close = []
    trade_df_atm_iv = []
    trade_df_atm_delta = []
    trade_df_atm_gamma = []
    trade_df_atm_theta = []
    trade_df_atm_vega = []
    trade_df_atm_qty = []
    trade_df_otm_put_strike = []
    trade_df_otm_put_price = []
    trade_df_otm_put_position_qty = []
    trade_df_otm_put_iv = []
    trade_df_otm_put_delta = []
    trade_df_otm_put_gamma = []
    trade_df_otm_put_theta = []
    trade_df_otm_put_vega = []
    trade_df_otm_call_strike = []
    trade_df_otm_call_price = []
    trade_df_otm_call_position_qty = []
    trade_df_otm_call_iv = []
    trade_df_otm_call_delta = []
    trade_df_otm_call_gamma = []
    trade_df_otm_call_theta = []
    trade_df_otm_call_vega = []
    trade_df_net_delta = []
    trade_df_futures_qty = []
    trade_df_futures_pnl = []
    for i in range(0, len(temp_df)):
        try:
            atm_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] == atm_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'put_close', 'call_close']].reset_index().drop('index', axis=1)
            otm_put_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] <= otm_put_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'put_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[False])      
            otm_call_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] >= otm_call_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'implied_volatility','delta', 'gamma', 'theta', 'vega', 'call_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[True])      
            trade_df_date.append(atm_delta_df['date'].iloc[0])
            trade_df_expiry.append(atm_delta_df['expiry'].iloc[0])
            trade_df_atm_strike.append(atm_delta_df['strike'].iloc[0])
            trade_df_forward_close.append(atm_delta_df['forward_close'].iloc[0])
            trade_df_atm_straddle_price.append(atm_delta_df['put_close'].iloc[0] + atm_delta_df['call_close'].iloc[0])            
            trade_df_atm_iv.append(atm_delta_df['implied_volatility'].iloc[0])
            trade_df_atm_delta.append(atm_delta_df['delta'].iloc[0])
            trade_df_atm_gamma.append(atm_delta_df['gamma'].iloc[0])
            trade_df_atm_theta.append(atm_delta_df['theta'].iloc[0])
            trade_df_atm_vega.append(atm_delta_df['vega'].iloc[0])
            trade_df_otm_put_strike.append(otm_put_delta_df['strike'].iloc[0])
            trade_df_otm_put_price.append(otm_put_delta_df['put_close'].iloc[0])
            trade_df_otm_put_iv.append(otm_put_delta_df['implied_volatility'].iloc[0])
            trade_df_otm_put_delta.append(otm_put_delta_df['delta'].iloc[0])
            trade_df_otm_put_gamma.append(otm_put_delta_df['gamma'].iloc[0])
            trade_df_otm_put_theta.append(otm_put_delta_df['theta'].iloc[0])
            trade_df_otm_put_vega.append(otm_put_delta_df['vega'].iloc[0])
            trade_df_otm_call_strike.append(otm_call_delta_df['strike'].iloc[0])
            trade_df_otm_call_price.append(otm_call_delta_df['call_close'].iloc[0])
            trade_df_otm_call_iv.append(otm_call_delta_df['implied_volatility'].iloc[0])
            trade_df_otm_call_delta.append(otm_call_delta_df['delta'].iloc[0])
            trade_df_otm_call_gamma.append(otm_call_delta_df['gamma'].iloc[0])
            trade_df_otm_call_theta.append(otm_call_delta_df['theta'].iloc[0])
            trade_df_otm_call_vega.append(otm_call_delta_df['vega'].iloc[0])
        except:
            atm_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'].between(atm_strike - 50, atm_strike + 50))
            ][['date', 'expiry', 'strike', 'forward_close', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'put_close', 'call_close']].reset_index().drop('index', axis=1)
            otm_put_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] <= otm_put_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'put_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[False])      
            otm_call_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] >= otm_call_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'implied_volatility','delta', 'gamma', 'theta', 'vega', 'call_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[True])      
            trade_df_date.append(atm_delta_df['date'].iloc[0])
            trade_df_expiry.append(atm_delta_df['expiry'].iloc[0])
            trade_df_atm_strike.append(atm_delta_df['strike'].iloc[0])
            trade_df_forward_close.append(atm_delta_df['forward_close'].iloc[0])
            trade_df_atm_straddle_price.append(atm_delta_df['put_close'].iloc[0] + atm_delta_df['call_close'].iloc[0])            
            trade_df_atm_iv.append(atm_delta_df['implied_volatility'].iloc[0])
            trade_df_atm_delta.append(atm_delta_df['delta'].iloc[0])
            trade_df_atm_gamma.append(atm_delta_df['gamma'].iloc[0])
            trade_df_atm_theta.append(atm_delta_df['theta'].iloc[0])
            trade_df_atm_vega.append(atm_delta_df['vega'].iloc[0])
            trade_df_otm_put_strike.append(otm_put_delta_df['strike'].iloc[0])
            trade_df_otm_put_price.append(otm_put_delta_df['put_close'].iloc[0])
            trade_df_otm_put_iv.append(otm_put_delta_df['implied_volatility'].iloc[0])
            trade_df_otm_put_delta.append(otm_put_delta_df['delta'].iloc[0])
            trade_df_otm_put_gamma.append(otm_put_delta_df['gamma'].iloc[0])
            trade_df_otm_put_theta.append(otm_put_delta_df['theta'].iloc[0])
            trade_df_otm_put_vega.append(otm_put_delta_df['vega'].iloc[0])
            trade_df_otm_call_strike.append(otm_call_delta_df['strike'].iloc[0])
            trade_df_otm_call_price.append(otm_call_delta_df['call_close'].iloc[0])
            trade_df_otm_call_iv.append(otm_call_delta_df['implied_volatility'].iloc[0])
            trade_df_otm_call_delta.append(otm_call_delta_df['delta'].iloc[0])
            trade_df_otm_call_gamma.append(otm_call_delta_df['gamma'].iloc[0])
            trade_df_otm_call_theta.append(otm_call_delta_df['theta'].iloc[0])
            trade_df_otm_call_vega.append(otm_call_delta_df['vega'].iloc[0])
            continue
    try:
        trade_df['date'] = trade_df_date
        trade_df['expiry'] = trade_df_expiry
        trade_df['atm_strike'] = trade_df_atm_strike
        trade_df['atm_straddle_price'] = trade_df_atm_straddle_price
        trade_df['forward_close'] = trade_df_forward_close
        trade_df['atm_iv'] = trade_df_atm_iv
        trade_df['atm_delta'] = trade_df_atm_delta
        trade_df['atm_gamma'] = trade_df_atm_gamma
        trade_df['atm_theta'] = trade_df_atm_theta
        trade_df['atm_vega'] = trade_df_atm_vega
        trade_df['otm_put_strike'] = trade_df_otm_put_strike
        trade_df['otm_put_price'] = trade_df_otm_put_price
        trade_df['otm_put_iv'] = trade_df_otm_put_iv
        trade_df['otm_put_delta'] = trade_df_otm_put_delta
        trade_df['otm_put_gamma'] = trade_df_otm_put_gamma
        trade_df['otm_put_theta'] = trade_df_otm_put_theta
        trade_df['otm_put_vega'] = trade_df_otm_put_vega
        trade_df['otm_call_strike'] = trade_df_otm_call_strike
        trade_df['otm_call_price'] = trade_df_otm_call_price
        trade_df['otm_call_iv'] = trade_df_otm_call_iv
        trade_df['otm_call_delta'] = trade_df_otm_call_delta
        trade_df['otm_call_gamma'] = trade_df_otm_call_gamma
        trade_df['otm_call_theta'] = trade_df_otm_call_theta
        trade_df['otm_call_vega'] = trade_df_otm_call_vega
        trade_df['atm_qty'] = atm_qty
        trade_df['otm_put_position_qty'] = otm_put_position_qty
        trade_df['otm_call_position_qty'] = otm_call_position_qty
        trade_df['atm_call_delta'] = np.where(trade_df['atm_delta'] >= 0, trade_df['atm_delta'], trade_df['atm_delta'] + 1)
        trade_df['atm_put_delta'] = np.where(trade_df['atm_delta'] <= 0, trade_df['atm_delta'], trade_df['atm_delta'] - 1)
        trade_df['otm_put_delta'] = np.where(trade_df['otm_put_delta'] <= 0, trade_df['otm_put_delta'], trade_df['otm_put_delta'] - 1)
        trade_df['otm_call_delta'] = np.where(trade_df['otm_call_delta'] >= 0, trade_df['otm_call_delta'], trade_df['otm_call_delta'] + 1)
        trade_df['position_delta'] = ''
        trade_df['net_delta'] = ''
        trade_df['futures_qty_to_buy_or_sell'] = ''
        trade_df['net_futures_qty_post_hedging'] = ''
        trade_df['net_delta_post_hedging'] = ''
        trade_df['futures_times_qty'] = ''
        trade_df['position_gamma'] = ''
        trade_df['position_theta'] = ''
        trade_df['position_vega'] = ''
        for i in range(0, len(trade_df)):
            if i == 0:
                trade_df['position_delta'][i] = -1*(trade_df['atm_call_delta'][i] + trade_df['atm_put_delta'][i]) * atm_qty + trade_df['otm_put_delta'][i] * otm_put_position_qty + trade_df['otm_call_delta'][i] * otm_call_position_qty           
                trade_df['net_delta'][i] = trade_df['position_delta'][i]
                trade_df['futures_qty_to_buy_or_sell'][i] = - trade_df['net_delta'][i]
                trade_df['net_futures_qty_post_hedging'][i] = trade_df['futures_qty_to_buy_or_sell'][i]
                trade_df['net_delta_post_hedging'][i] = trade_df['futures_qty_to_buy_or_sell'][i] + trade_df['net_delta'][i]
                trade_df['position_gamma'][i] = -1*(trade_df['atm_gamma'][i] * 2) * atm_qty + trade_df['otm_put_gamma'][i] * otm_put_position_qty + trade_df['otm_call_gamma'][i] * otm_call_position_qty
                trade_df['position_theta'][i] = -1*(trade_df['atm_theta'][i] * 2) * atm_qty + trade_df['otm_put_theta'][i] * otm_put_position_qty + trade_df['otm_call_theta'][i] * otm_call_position_qty
                trade_df['position_vega'][i] = -1*(trade_df['atm_vega'][i] * 2) * atm_qty + trade_df['otm_put_vega'][i] * otm_put_position_qty + trade_df['otm_call_vega'][i] * otm_call_position_qty
            else:
                trade_df['position_delta'][i] = -1*(trade_df['atm_call_delta'][i] + trade_df['atm_put_delta'][i]) * atm_qty + trade_df['otm_put_delta'][i] * otm_put_position_qty + trade_df['otm_call_delta'][i] * otm_call_position_qty           
                trade_df['net_delta'][i] = trade_df['position_delta'][i] + trade_df['net_futures_qty_post_hedging'][i-1]
                trade_df['futures_qty_to_buy_or_sell'][i] = - trade_df['net_delta'][i]
                trade_df['net_futures_qty_post_hedging'][i] = trade_df['futures_qty_to_buy_or_sell'][i] + trade_df['net_futures_qty_post_hedging'][i-1]
                trade_df['net_delta_post_hedging'][i] = trade_df['futures_qty_to_buy_or_sell'][i] + trade_df['net_delta'][i]
                trade_df['position_gamma'][i] = -1*(trade_df['atm_gamma'][i] * 2) * atm_qty + trade_df['otm_put_gamma'][i] * otm_put_position_qty + trade_df['otm_call_gamma'][i] * otm_call_position_qty
                trade_df['position_theta'][i] = -1*(trade_df['atm_theta'][i] * 2) * atm_qty + trade_df['otm_put_theta'][i] * otm_put_position_qty + trade_df['otm_call_theta'][i] * otm_call_position_qty
                trade_df['position_vega'][i] = -1*(trade_df['atm_vega'][i] * 2) * atm_qty + trade_df['otm_put_vega'][i] * otm_put_position_qty + trade_df['otm_call_vega'][i] * otm_call_position_qty
            trade_df['futures_times_qty'][i] = trade_df['forward_close'][i] * trade_df['futures_qty_to_buy_or_sell'][i]    
        atm_pnl = - (trade_df['atm_straddle_price'].iloc[len(trade_df) - 1] - trade_df['atm_straddle_price'].iloc[0]) * trade_df['atm_qty'].iloc[0]
        otm_put_pnl = (trade_df['otm_put_price'].iloc[len(trade_df) - 1] - trade_df['otm_put_price'].iloc[0]) * trade_df['otm_put_position_qty'].iloc[0]
        otm_call_pnl = (trade_df['otm_call_price'].iloc[len(trade_df) - 1] - trade_df['otm_call_price'].iloc[0]) * trade_df['otm_call_position_qty'].iloc[0]
        options_pnl = atm_pnl + otm_put_pnl + otm_call_pnl
        futures_pnl = trade_df['futures_times_qty'].sum() + (-1 * trade_df['net_futures_qty_post_hedging'].iloc[len(trade_df) - 1] * trade_df['forward_close'].iloc[len(trade_df) - 1])
        net_pnl = futures_pnl + options_pnl
        date_list.append(trade_df['date'].iloc[len(trade_df)-1])
        options_pnl_list.append(options_pnl)
        futures_pnl_list.append(futures_pnl)
        net_pnl_list.append(net_pnl)
        delta_adjustments_df = pd.concat([delta_adjustments_df, trade_df], axis=0)
        daily_greek_exposure_df = pd.concat([daily_greek_exposure_df, trade_df], axis=0)
    except:
        continue
delta_adjustments_df = delta_adjustments_df.reset_index().drop('index', axis=1)
daily_greek_exposure_df = daily_greek_exposure_df.reset_index().drop('index', axis=1)
short_pnl_df['date'] = date_list    
short_pnl_df['options_pnl'] = options_pnl_list
short_pnl_df['futures_pnl'] = futures_pnl_list
short_pnl_df['atm_pnl'] = net_pnl_list
short_pnl_df['cumulative_atm_pnl'] = short_pnl_df['atm_pnl'].cumsum()

# Make 'short_pnl_df' compatible with other pnl_df's
short_pnl_df['date'] = pd.to_datetime(short_pnl_df['date'], format='mixed')
short_pnl_df['month'] = short_pnl_df['date'].apply(lambda x: "{}-{}".format(x.month_name()[:3],x.year))
monthly_atm_pnl = short_pnl_df.groupby(['month'], sort=False)['atm_pnl'].sum()
monthly_short_pnl_df = pd.DataFrame(monthly_atm_pnl)
monthly_short_pnl_df = monthly_short_pnl_df.reset_index()
monthly_short_pnl_df['atm_pnl'] = monthly_short_pnl_df['atm_pnl'].fillna(0)
monthly_short_pnl_df['cumulative_atm_pnl'] = monthly_short_pnl_df['atm_pnl'].cumsum()

In [37]:
# Entry & exit date for each short volatility trade
entry_df = pd.DataFrame(atm_otm_improved_df.groupby('trade_group').first()).reset_index()
entry_df = entry_df[
    (entry_df['trade_group'] != 0)
].reset_index().drop('index', axis=1)[[
    'trade_group', 'date', 'atm_strike', 'otm_put_strike', 'atm_iv', 'realized_vol',
    'vol_risk_premium', 'otm_call_strike', 'position_delta', 'position_gamma', 
    'position_theta', 'position_vega', 'gamma_risk', 'vega_risk'
]].rename(columns={
    'date': 'entry_date'
})
exit_df = pd.DataFrame(atm_otm_improved_df.groupby('trade_group').last()).reset_index()
exit_df = exit_df[
    (exit_df['trade_group'] != 0)
].reset_index().drop('index', axis=1)[[
    'trade_group', 'date'
]].rename(columns={
    'date': 'exit_date'
})
entry_exit_short_df = entry_df.merge(exit_df, on=['trade_group'], how='inner')
entry_exit_short_df = entry_exit_short_df.rename(columns={
    'trade_group': 'trade_id'
})[[
    'trade_id', 'entry_date', 'exit_date', 'atm_iv', 'realized_vol',
    'vol_risk_premium', 'atm_strike', 'otm_put_strike', 'otm_call_strike', 
    'position_delta', 'position_gamma', 'position_theta', 'position_vega',
    'gamma_risk', 'vega_risk'
]]
entry_exit_short_df['gamma_risk'] = round(entry_exit_short_df['gamma_risk'])
entry_exit_short_df['vega_risk'] = round(entry_exit_short_df['vega_risk'])

def convert_to_percentage(x):
    return "{:.2%}".format(x)

entry_exit_short_df['atm_iv'] = entry_exit_short_df['atm_iv'].apply(convert_to_percentage)
entry_exit_short_df['realized_vol'] = entry_exit_short_df['realized_vol'].apply(convert_to_percentage)
entry_exit_short_df['vol_risk_premium'] = entry_exit_short_df['vol_risk_premium'].apply(convert_to_percentage)
entry_exit_short_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/entry_exit_short_df.csv')
entry_exit_short_df

,trade_id,entry_date,exit_date,atm_iv,realized_vol,vol_risk_premium,atm_strike,otm_put_strike,otm_call_strike,position_delta,position_gamma,position_theta,position_vega,gamma_risk,vega_risk
0,2,2016-01-18,2016-01-19,19.09%,15.10%,26.39%,7400.0,6800.0,7900.0,-8.620408,-1.247474,3062.78,-13007.82,-3080.0,-48395.0
1,3,2016-01-21,2016-01-22,19.14%,15.35%,24.68%,7300.0,6700.0,7800.0,-59.561068,-1.377485,3101.70,-12657.12,-3434.0,-47223.0
2,9,2016-06-22,2016-06-29,16.30%,10.98%,48.52%,8200.0,7400.0,9000.0,-134.286419,-1.732719,3685.56,-17783.52,-2810.0,-56518.0
3,11,2016-07-25,2016-08-05,14.13%,9.37%,50.83%,8700.0,7800.0,9400.0,33.735348,-2.227360,4087.92,-19197.00,-2925.0,-52884.0
4,12,2016-08-08,2016-08-11,12.84%,9.45%,35.92%,8750.0,7900.0,9300.0,25.838402,-3.297357,5215.76,-14581.82,-4461.0,-36500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,181,2022-11-17,2022-11-22,12.98%,9.55%,35.91%,18500.0,16600.0,20000.0,4.442936,-0.401645,2918.70,-19765.70,-2481.0,-49998.0
98,182,2022-12-09,2022-12-15,11.21%,7.82%,43.31%,18600.0,16700.0,21000.0,8.760594,-0.784282,4238.45,-16085.60,-3288.0,-35147.0
99,184,2022-12-16,2022-12-19,13.02%,9.74%,33.57%,18450.0,16700.0,20000.0,-0.987687,-0.410606,2986.15,-19167.35,-2629.0,-48627.0
100,185,2022-12-20,2022-12-26,12.83%,9.62%,33.35%,18500.0,17000.0,19700.0,-25.889103,-0.399874,2848.90,-16770.50,-2519.0,-41930.0


In [47]:
# Merge entry_exit_short_df with that of equal vega risk and equal gamma risk scenarios
path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/entry_exit_short_df_eq_v.csv'
eq_v_df = pd.read_csv(path)
path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/entry_exit_short_df_eq_g.csv'
eq_g_df = pd.read_csv(path)
eq_v_df = eq_v_df.drop('Unnamed: 0', axis=1)
eq_g_df = eq_g_df.drop('Unnamed: 0', axis=1)
eq_g_df
entry_exit_short_df = entry_exit_short_df.merge(eq_v_df[[
    'trade_id', 'vega_risk_eq_v', 'scaling_factor_eq_v'
]], on=['trade_id'], how='inner')
entry_exit_short_df = entry_exit_short_df.merge(eq_g_df[[    
    'trade_id', 'gamma_risk_eq_g', 'scaling_factor_eq_g'
]], on=['trade_id'], how='inner')
entry_exit_short_df = entry_exit_short_df.rename(columns={
    'vega_risk_eq_v': 'vega_risk_eq_vega',
    'scaling_factor_eq_v': 'scaling_factor_eq_vega',
    'gamma_risk_eq_g': 'vega_risk_eq_gamma',
    'scaling_factor_eq_g': 'scaling_factor_eq_gamma'
})
entry_exit_short_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/entry_exit_short_df.csv')

In [29]:
# Delta adjustments for the short vol trades
delta_adjustments_df = pd.DataFrame(delta_adjustments_df.merge(atm_otm_improved_df[['date', 'expiry', 'trade_group']], on=['date', 'expiry'], how='left'))
delta_adjustments_df = delta_adjustments_df[[
    'trade_group', 'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close',
    'atm_delta', 'atm_qty', 'atm_call_delta', 'atm_put_delta', 'otm_put_strike',
    'otm_put_price', 'otm_put_position_qty', 'otm_put_delta', 'otm_call_strike', 
    'otm_call_price', 'otm_call_position_qty', 'otm_call_delta',
    'position_delta', 'net_delta', 'futures_qty_to_buy_or_sell',
    'net_futures_qty_post_hedging', 'net_delta_post_hedging'
]].rename(columns={
    'trade_group': 'trade_id',
    'otm_put_position_qty': 'otm_put_qty',
    'otm_call_position_qty': 'otm_call_qty'
})
delta_adjustments_df['otm_put_qty'] = round(delta_adjustments_df['otm_put_qty'], 0)
delta_adjustments_df['otm_call_qty'] = round(delta_adjustments_df['otm_call_qty'], 0)
delta_adjustments_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/delta_adjustments_df.csv')
delta_adjustments_df

,trade_id,date,expiry,atm_strike,atm_straddle_price,forward_close,atm_delta,atm_qty,atm_call_delta,atm_put_delta,...,otm_put_delta,otm_call_strike,otm_call_price,otm_call_qty,otm_call_delta,position_delta,net_delta,futures_qty_to_buy_or_sell,net_futures_qty_post_hedging,net_delta_post_hedging
0,2,2016-01-18,2016-02-25,7400.0,361.75,7386.55,0.498091,1400.0,0.498091,-0.501909,...,-0.127050,7900.0,22.90,1400.0,0.117075,-8.620408,-8.620408,8.620408,8.620408,0.0
1,2,2016-01-19,2016-02-25,7400.0,335.40,7453.45,-0.437034,1400.0,0.562966,-0.437034,...,-0.093251,7900.0,21.50,1400.0,0.121837,-136.285894,-127.665486,127.665486,136.285894,0.0
2,3,2016-01-21,2016-02-25,7300.0,344.05,7301.55,-0.485101,1400.0,0.514899,-0.485101,...,-0.124425,7800.0,20.55,1400.0,0.111680,-59.561068,-59.561068,59.561068,59.561068,0.0
3,3,2016-01-22,2016-02-25,7300.0,341.85,7439.05,-0.355716,1400.0,0.644284,-0.355716,...,-0.069736,7800.0,31.45,1400.0,0.168240,-266.090256,-206.529188,206.529188,266.090256,0.0
4,9,2016-06-22,2016-07-28,8200.0,335.75,8235.75,-0.454208,1200.0,0.545792,-0.454208,...,-0.063795,9000.0,7.35,1200.0,0.043474,-134.286419,-134.286419,134.286419,134.286419,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,187,2023-01-27,2023-02-23,18000.0,683.15,17681.60,0.352228,600.0,0.352228,-0.647772,...,-0.074942,19500.0,9.25,600.0,0.027561,148.898391,147.954655,-147.954655,-148.898391,0.0
559,187,2023-01-30,2023-02-23,18000.0,644.05,17740.80,0.373411,600.0,0.373411,-0.626589,...,-0.064372,19500.0,8.25,600.0,0.025889,128.81758,-20.080812,20.080812,-128.81758,0.0
560,187,2023-01-31,2023-02-23,18000.0,590.80,17798.90,0.394506,600.0,0.394506,-0.605494,...,-0.045944,19500.0,6.20,600.0,0.021364,111.844646,-16.972933,16.972933,-111.844646,0.0
561,187,2023-02-01,2023-02-23,18000.0,601.75,17671.95,0.319472,600.0,0.319472,-0.680528,...,-0.061303,19500.0,5.00,600.0,0.017277,190.21856,78.373914,-78.373914,-190.21856,0.0


In [30]:
# Delta adjustments for the short vol trades
daily_greek_exposure_df = pd.DataFrame(daily_greek_exposure_df.merge(atm_otm_improved_df[['date', 'expiry', 'trade_group']], on=['date', 'expiry'], how='left'))
daily_greek_exposure_df = daily_greek_exposure_df[[
    'trade_group', 'date', 'expiry', 'forward_close', 'atm_strike', 'atm_qty', 'atm_vega',
    'otm_put_strike', 'otm_put_position_qty', 'otm_put_vega', 'otm_call_strike', 
    'otm_call_position_qty', 'otm_call_vega', 
    'atm_iv', 'otm_put_iv', 'otm_call_iv', 
    'net_delta_post_hedging', 'position_gamma', 'position_theta', 'position_vega'
]].rename(columns={
    'trade_group': 'trade_id',
    'otm_put_position_qty': 'otm_put_qty',
    'otm_call_position_qty': 'otm_call_qty'
})
daily_greek_exposure_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/daily_greek_exposure_df.csv')
daily_greek_exposure_df

,trade_id,date,expiry,forward_close,atm_strike,atm_qty,atm_vega,otm_put_strike,otm_put_qty,otm_put_vega,otm_call_strike,otm_call_qty,otm_call_vega,atm_iv,otm_put_iv,otm_call_iv,net_delta_post_hedging,position_gamma,position_theta,position_vega
0,2,2016-01-18,2016-02-25,7386.55,7400.0,1400.0,9.4624,6800.0,1400.0,4.9549,7900.0,1400.0,4.6786,0.190869,0.233137,0.171486,0.0,-1.247474,3062.78,-13007.82
1,2,2016-01-19,2016-02-25,7453.45,7400.0,1400.0,9.3300,6800.0,1400.0,3.9538,7900.0,1400.0,4.7946,0.177233,0.224464,0.153723,0.0,-1.404142,3191.16,-13876.24
2,3,2016-01-21,2016-02-25,7301.55,7300.0,1400.0,8.9846,6700.0,1400.0,4.6346,7800.0,1400.0,4.2938,0.191406,0.249590,0.171672,0.0,-1.377485,3101.7,-12657.12
3,3,2016-01-22,2016-02-25,7439.05,7300.0,1400.0,8.4465,6700.0,1400.0,3.0381,7800.0,1400.0,5.7026,0.181803,0.238036,0.157809,0.0,-1.195517,2988.02,-11413.22
4,9,2016-06-22,2016-07-28,8235.75,8200.0,1200.0,10.2158,7400.0,1200.0,3.2294,9000.0,1200.0,2.3826,0.163046,0.229337,0.162818,0.0,-1.732719,3685.56,-17783.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,187,2023-01-27,2023-02-23,17681.60,18000.0,600.0,17.8035,16300.0,600.0,6.7963,19500.0,600.0,3.0472,0.165041,0.212344,0.185407,0.0,-0.434056,4317.24,-15458.1
559,187,2023-01-30,2023-02-23,17740.80,18000.0,600.0,17.1660,16300.0,600.0,5.7181,19500.0,600.0,2.7354,0.166813,0.221996,0.187472,0.0,-0.479719,4956.72,-15527.1
560,187,2023-01-31,2023-02-23,17798.90,18000.0,600.0,17.1013,16300.0,600.0,4.2998,19500.0,600.0,2.2849,0.159990,0.211717,0.177770,0.0,-0.54286,5462.28,-16570.74
561,187,2023-02-01,2023-02-23,17671.95,18000.0,600.0,15.4762,16300.0,600.0,5.2522,19500.0,600.0,1.8538,0.154375,0.217177,0.187770,0.0,-0.530545,4499.88,-14307.84


In [31]:
# Daily PnL - Greeks Attribution
trade_groups = list(set(atm_otm_improved_df['trade_group'].unique()) - set([0]))
temp_pnl_df = pd.DataFrame(columns=['trade_id', 'date', 'daily_delta_pnl', 'daily_gamma_pnl', 'daily_theta_pnl', 'daily_vega_pnl'])
for tg in trade_groups:
    temp_df = daily_greek_exposure_df[
        (daily_greek_exposure_df['trade_id'] == tg)
    ]
    temp_df['date'] = pd.to_datetime(temp_df['date'], format='mixed')
    temp_df['daily_delta_pnl'] = temp_df['net_delta_post_hedging'].shift(1) * (temp_df['forward_close'] - temp_df['forward_close'].shift(1))
    temp_df['daily_gamma_pnl'] = 0.5 * temp_df['position_gamma'].shift(1) * (temp_df['forward_close'] - temp_df['forward_close'].shift(1))**2
    temp_df['daily_theta_pnl'] = temp_df['position_theta'].shift(1) * (temp_df['date'] - temp_df['date'].shift(1)).dt.days
    atm_iv_diff = temp_df['atm_iv'] - temp_df['atm_iv'].shift(1)
    otm_put_iv_diff = temp_df['otm_put_iv'] - temp_df['otm_put_iv'].shift(1)
    otm_call_iv_diff = temp_df['otm_call_iv'] - temp_df['otm_call_iv'].shift(1)
    temp_df['daily_vega_pnl'] = 100.00 * (- temp_df['atm_qty'] * temp_df['atm_vega'].shift(1) * atm_iv_diff * 2 + temp_df['otm_put_qty'] * temp_df['otm_put_vega'].shift(1) * otm_put_iv_diff  + temp_df['otm_call_qty'] * temp_df['otm_call_vega'].shift(1) * otm_call_iv_diff)       
    temp_df = temp_df[['trade_id', 'date', 'daily_delta_pnl', 'daily_gamma_pnl', 'daily_theta_pnl', 'daily_vega_pnl']]
    temp_pnl_df = pd.concat([temp_pnl_df, temp_df], axis = 0)
daily_greek_wise_pnl_df = temp_pnl_df
daily_greek_wise_pnl_df = daily_greek_wise_pnl_df.fillna(0)

In [55]:
temp_df_01 = pd.DataFrame(delta_adjustments_df[[
    'trade_id', 'expiry', 'forward_close', 'atm_qty', 'atm_straddle_price',
    'otm_put_qty', 'otm_put_price', 'otm_call_qty', 'otm_call_price'
]].groupby('trade_id').first()).reset_index()
temp_df_02 = pd.DataFrame(entry_exit_short_df.merge(temp_df_01, on=['trade_id'], how='inner'))
temp_df_02['exposure'] = round(temp_df_02['forward_close'] * temp_df_02['atm_qty'], -6)
short_pnl_df = short_pnl_df.rename(columns={
    'date': 'exit_date'
})
temp_df_02['exit_date'] = pd.to_datetime(temp_df_02['exit_date'])
trades_pnl_df = temp_df_02.merge(short_pnl_df[['exit_date', 'options_pnl', 'futures_pnl', 'atm_pnl', 'cumulative_atm_pnl']], on=['exit_date'], how='inner')
trades_pnl_df['otm_put_qty'] = round(trades_pnl_df['otm_put_qty'], 0)
trades_pnl_df['otm_call_qty'] = round(trades_pnl_df['otm_call_qty'], 0)
trades_pnl_df = trades_pnl_df.rename(columns={
    'atm_pnl': 'pnl',
    'cumulative_atm_pnl': 'cumulative_pnl'
})
trades_pnl_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/trades_pnl_df.csv')
trades_pnl_df

,trade_id,entry_date,exit_date,atm_iv,realized_vol,vol_risk_premium,atm_strike,otm_put_strike,otm_call_strike,position_delta,...,atm_straddle_price,otm_put_qty,otm_put_price,otm_call_qty,otm_call_price,exposure,options_pnl,futures_pnl,pnl,cumulative_pnl
0,2,2016-01-18,2016-01-19,19.09%,15.10%,26.39%,7400.0,6800.0,7900.0,-8.620408,...,361.75,1400.0,36.40,1400.0,22.90,10000000.0,17430.0,-576.705294,16853.294706,1.685329e+04
1,3,2016-01-21,2016-01-22,19.14%,15.35%,24.68%,7300.0,6700.0,7800.0,-59.561068,...,344.05,1400.0,36.05,1400.0,20.55,10000000.0,-8120.0,-8189.646871,-16309.646871,5.436478e+02
2,9,2016-06-22,2016-06-29,16.30%,10.98%,48.52%,8200.0,7400.0,9000.0,-134.286419,...,335.75,1200.0,16.95,1200.0,7.35,10000000.0,53700.0,48176.346134,101876.346134,1.024200e+05
3,11,2016-07-25,2016-08-05,14.13%,9.37%,50.83%,8700.0,7800.0,9400.0,33.735348,...,284.50,1200.0,8.15,1200.0,5.40,10000000.0,82380.0,38291.838227,120671.838227,2.230918e+05
4,12,2016-08-08,2016-08-11,12.84%,9.45%,35.92%,8750.0,7900.0,9300.0,25.838402,...,192.90,1100.0,3.05,1100.0,1.75,10000000.0,-6930.0,-11503.857210,-18433.857210,2.046580e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,181,2022-11-17,2022-11-22,12.98%,9.55%,35.91%,18500.0,16600.0,20000.0,4.442936,...,652.40,500.0,17.85,500.0,11.25,9000000.0,42225.0,5513.646254,47738.646254,9.822514e+05
98,182,2022-12-09,2022-12-15,11.21%,7.82%,43.31%,18600.0,16700.0,21000.0,8.760594,...,387.70,500.0,3.75,500.0,2.65,9000000.0,21950.0,33559.822953,55509.822953,1.037761e+06
99,184,2022-12-16,2022-12-19,13.02%,9.74%,33.57%,18450.0,16700.0,20000.0,-0.987687,...,628.45,500.0,18.20,500.0,10.60,9000000.0,3375.0,-161.190456,3213.809544,1.040975e+06
100,185,2022-12-20,2022-12-26,12.83%,9.62%,33.35%,18500.0,17000.0,19700.0,-25.889103,...,591.00,500.0,23.60,500.0,14.60,9000000.0,-14875.0,7492.672086,-7382.327914,1.033593e+06


In [56]:
# Merge trades_pnl_df with that of equal vega risk and equal gamma risk scenarios
trades_pnl_df = trades_pnl_df.rename(columns={
    'pnl': 'pnl_1cr_exposure',
    'cumulative_pnl': 'cumulative_pnl_1cr_exposure'
})
path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/trades_pnl_df_eq_v.csv'
eq_v_df = pd.read_csv(path)
path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/trades_pnl_df_eq_g.csv'
eq_g_df = pd.read_csv(path)
eq_v_df
eq_v_df = eq_v_df.drop('Unnamed: 0', axis=1)
eq_g_df = eq_g_df.drop('Unnamed: 0', axis=1)
eq_v_df = eq_v_df.rename(columns={
    'pnl': 'pnl_equal_vega',
    'cumulative_pnl': 'cumulative_pnl_equal_vega'
})
eq_g_df = eq_g_df.rename(columns={
    'pnl': 'pnl_equal_gamma',
    'cumulative_pnl': 'cumulative_pnl_equal_gamma'
})
trades_pnl_df = trades_pnl_df.merge(eq_v_df[[
    'trade_id', 'pnl_equal_vega', 'cumulative_pnl_equal_vega'
]], on=['trade_id'], how='inner')
trades_pnl_df = trades_pnl_df.merge(eq_g_df[[
    'trade_id', 'pnl_equal_gamma', 'cumulative_pnl_equal_gamma'
]], on=['trade_id'], how='inner')
trades_pnl_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/trades_pnl_df.csv')
trades_pnl_df

,trade_id,entry_date,exit_date,atm_iv,realized_vol,vol_risk_premium,atm_strike,otm_put_strike,otm_call_strike,position_delta,...,otm_call_price,exposure,options_pnl,futures_pnl,pnl_1cr_exposure,cumulative_pnl_1cr_exposure,pnl_equal_vega,cumulative_pnl_equal_vega,pnl_equal_gamma,cumulative_pnl_equal_gamma
0,2,2016-01-18,2016-01-19,19.09%,15.10%,26.39%,7400.0,6800.0,7900.0,-8.620408,...,22.90,10000000.0,17430.0,-576.705294,16853.294706,1.685329e+04,17407.045818,17407.045818,37110.469639,3.711047e+04
1,3,2016-01-21,2016-01-22,19.14%,15.35%,24.68%,7300.0,6700.0,7800.0,-59.561068,...,20.55,10000000.0,-8120.0,-8189.646871,-16309.646871,5.436478e+02,-17264.926188,142.119630,-32210.148829,4.900321e+03
2,9,2016-06-22,2016-06-29,16.30%,10.98%,48.52%,8200.0,7400.0,9000.0,-134.286419,...,7.35,10000000.0,53700.0,48176.346134,101876.346134,1.024200e+05,90160.566328,90302.685958,245837.352857,2.507377e+05
3,11,2016-07-25,2016-08-05,14.13%,9.37%,50.83%,8700.0,7800.0,9400.0,33.735348,...,5.40,10000000.0,82380.0,38291.838227,120671.838227,2.230918e+05,114135.446989,204438.132948,279804.161294,5.305418e+05
4,12,2016-08-08,2016-08-11,12.84%,9.45%,35.92%,8750.0,7900.0,9300.0,25.838402,...,1.75,10000000.0,-6930.0,-11503.857210,-18433.857210,2.046580e+05,-25254.384377,179183.748570,-28019.881842,5.025220e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,181,2022-11-17,2022-11-22,12.98%,9.55%,35.91%,18500.0,16600.0,20000.0,4.442936,...,11.25,9000000.0,42225.0,5513.646254,47738.646254,9.822514e+05,47738.646254,835803.548921,130501.236277,3.045832e+06
98,182,2022-12-09,2022-12-15,11.21%,7.82%,43.31%,18600.0,16700.0,21000.0,8.760594,...,2.65,9000000.0,21950.0,33559.822953,55509.822953,1.037761e+06,78934.968239,914738.517160,114493.338515,3.160325e+06
99,184,2022-12-16,2022-12-19,13.02%,9.74%,33.57%,18450.0,16700.0,20000.0,-0.987687,...,10.60,9000000.0,3375.0,-161.190456,3213.809544,1.040975e+06,3303.796211,918042.313370,8288.955261,3.168614e+06
100,185,2022-12-20,2022-12-26,12.83%,9.62%,33.35%,18500.0,17000.0,19700.0,-25.889103,...,14.60,9000000.0,-14875.0,7492.672086,-7382.327914,1.033593e+06,-8799.734873,909242.578497,-19874.352420,3.148739e+06


In [33]:
# Matching Greek-wise PnL with the final PnL
temp = daily_greek_wise_pnl_df.groupby(['trade_id'])[[
    'daily_delta_pnl', 'daily_gamma_pnl', 'daily_theta_pnl', 'daily_vega_pnl'
]].sum()
total_delta_pnl = temp['daily_delta_pnl'].sum()
total_gamma_pnl = temp['daily_gamma_pnl'].sum()
total_theta_pnl = temp['daily_theta_pnl'].sum()
total_vega_pnl = temp['daily_vega_pnl'].sum()
print(total_delta_pnl, total_gamma_pnl, total_theta_pnl, total_vega_pnl)
print(total_delta_pnl + total_gamma_pnl + total_theta_pnl + total_vega_pnl)
print(trades_pnl_df['cumulative_pnl'].iloc[-1])

0.0 -2690085.2304870314 2777915.980000001 753097.2061630223
840927.9556759918
1051036.6086667867


In [34]:
daily_greek_wise_pnl_df.loc['total_pnl'] = daily_greek_wise_pnl_df[[
    'daily_delta_pnl', 'daily_gamma_pnl', 'daily_theta_pnl', 'daily_vega_pnl'
]].sum()
daily_greek_wise_pnl_df = daily_greek_wise_pnl_df.fillna(0)
daily_greek_wise_pnl_df.iloc[-1].sum()
daily_greek_wise_pnl_df[[
    'daily_delta_pnl', 'daily_gamma_pnl', 'daily_theta_pnl', 'daily_vega_pnl'
]].iloc[-1].sum()
daily_greek_wise_pnl_df['daily_pnl'] = daily_greek_wise_pnl_df[[
    'daily_delta_pnl', 'daily_gamma_pnl', 'daily_theta_pnl', 'daily_vega_pnl'
]].sum(axis=1)
daily_greek_wise_pnl_df.to_csv('/Users/vamsikrishnasb/My Drive/Financial Analysis/Trader Job Interviews/Companies/Alpha Alternatives/Assignment/May 9/daily_greek_wise_pnl_df.csv')
daily_greek_wise_pnl_df

,trade_id,date,daily_delta_pnl,daily_gamma_pnl,daily_theta_pnl,daily_vega_pnl,daily_pnl
0,2.0,2016-01-18 00:00:00,0.0,0.000000e+00,0.00,0.000000,0.000000
1,2.0,2016-01-19 00:00:00,0.0,-2.791604e+03,3062.78,18478.702275,18749.878273
2,3.0,2016-01-21 00:00:00,0.0,0.000000e+00,0.00,0.000000,0.000000
3,3.0,2016-01-22 00:00:00,0.0,-1.302154e+04,3101.70,8328.454699,-1591.382435
4,9.0,2016-06-22 00:00:00,0.0,0.000000e+00,0.00,0.000000,0.000000
...,...,...,...,...,...,...,...
559,187.0,2023-01-30 00:00:00,0.0,-7.606048e+02,12951.72,527.649681,12718.764847
560,187.0,2023-01-31 00:00:00,0.0,-8.096722e+02,4956.72,8935.067080,13082.114919
561,187.0,2023-02-01 00:00:00,-0.0,-4.374450e+03,5462.28,14303.649679,15391.480080
562,187.0,2023-02-02 00:00:00,0.0,-3.916018e+01,4499.88,19508.544273,23969.264091


In [56]:
# Standard deviation of 1-day logarithmic changes in 1-month implied volatility 
# "vol_of_vol" or "vol_of_iv"
def pnl_analysis():
#     print(trades_pnl_df)
    trades_pnl_df['log_pnl_returns'] = np.log(trades_pnl_df['cumulative_pnl'] / trades_pnl_df['cumulative_pnl'].shift(1))
    trades_pnl_df['vol_of_pnl_returns'] = trades_pnl_df['log_pnl_returns'].rolling(30).std()
    return vol_of_pnl_returns

vol_of_pnl_returns = pnl_analysis()

In [55]:
fig = px.line(trades_pnl_df[40:], y='vol_of_pnl_returns')
fig.show()

In [36]:
fig = px.line(trades_pnl_df, y='cumulative_pnl')
fig.show()
# trades_pnl_df